In [15]:
from openai import AzureOpenAI
import time

import os 

import html
import io
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

In [11]:
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
# from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
# from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile

In [12]:
#load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
api_base = os.environ['api_base']
api_key = os.environ['api_key']
deployment_name = os.environ['deployment_name']

In [14]:
client = AzureOpenAI(
    api_key= api_key, #os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint=api_base # os.getenv("AZURE_OPENAI_ENDPOINT")
)


# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="you are a helpful assisatant that can help with math problems and other coding skills",
    tools=[{"type": "code_interpreter"}],
    
    model=deployment_name
)


In [16]:
DATA_FOLDER = "data/"


def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")


arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

In [17]:
file_ids

['assistant-6kT9y3OcEUhwzyXkJViAv2m8']

In [23]:
for f in client.files.list():
    print(f.id)
    print(f.filename)


assistant-6kT9y3OcEUhwzyXkJViAv2m8
sales.csv
assistant-vlJmh7YUR6M8pWYy4Dxt0m0p
sales.txt.txt


In [26]:
for a in client.beta.assistants.list():
    print(a.id)
    print(a.name)
    print(a.file_ids)

    

asst_yDiYPSx7cUNuLkZeCgIsvykI
Math Assist
[]
asst_fJXQSaNgL0d14ATuOLwPqGcJ
Math Assist
[]
asst_muTxOw5OnxqyynS9OW8wkMcB
Math Assist
[]
asst_sVWtfaAUTitZE9RYKpOJA6bc
Math Assist
[]
asst_CzqNVyUQWDmDp3V3YyXXOUv9
Math Assist
[]
asst_2yYIiMY2egSw02b6wHw4padW
Math Assist
[]
asst_F5gKwGkysI4OSeDWaGhwGy8n
salesdata
['assistant-vlJmh7YUR6M8pWYy4Dxt0m0p']


Assistant(id='asst_yDiYPSx7cUNuLkZeCgIsvykI', created_at=1711083147, description=None, file_ids=[], instructions='you are a helpful assisatant that can help with math problems and other coding skills', metadata={}, model='gpt-4-default-aueast', name='Math Assist', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')])


In [ ]:

def my_python_tool(

    input1: str, 
    aoaiconn : AzureOpenAIConnection, 
    deployment_name: str = "gpt-4-default-aueast",
    ) -> str:

    print(aoaiconn.api_base)

    print(aoaiconn.api_key)

    # rendered_prompt = Template(prompt, trim_blocks=True, keep_trailing_newline=True).render(**kwargs)




    # Create an assistant
    assistant = client.beta.assistants.create(
        name="Math Assist",
        instructions=input1,
        tools=[{"type": "code_interpreter"}],
        model=deployment_name
    )



    # add a file to assistant 
    # client.beta.assistants.files.create(
    #     assistant.id, 

    # )

    # Create a thread
    thread = client.beta.threads.create()

    # Add a user question to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
    )

    # Run the thread
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Retrieve the status of the run
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    status = run.status

    # Wait till the assistant has responded
    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        status = run.status

    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )


# Prints the messages with the latest message at the bottom
    number_of_messages = len(messages.data)
    print( f'Number of messages: {number_of_messages}')

    assistant_response = ''
    for message in reversed(messages.data):
        role = message.role  
        for content in message.content:
            if content.type == 'text':
                response = content.text.value 
                print(f'\n{role}: {response}')
                if role == 'assistant':
                    assistant_response = response  
    
    return assistant_response
